In [ ]:
% matplotlib inline
from __future__ import print_function

from collections import Counter
import json
import os

import pandas as pd
import lda
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse
from sklearn.decomposition import LatentDirichletAllocation

<h2>データを読み込もう</h2>

In [ ]:
with open(os.path.normpath('./dataset/document_word_data.json'), 'r') as f:
    doc_data = json.load(f)

この読み込んだjsonデータはこんな感じです。

In [ ]:
contents_data = []
for idx in doc_data.keys():
    each_doc = ' '.join(doc_data[idx])
    contents_data.append(each_doc)

この後、shuffleするのでNumPyの配列にしておきましょう.

In [ ]:
contents_data = np.array(contents_data)

単語のインデックスを作るために、全ての単語のリストを作ります。

###  CountVectorizerを用いてdoc_dataから疎行列を作ります

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def tokenizer(s):
    return s.split()

vectorizer = CountVectorizer(tokenizer=tokenizer)
vectorizer.fit(contents_data)
A = vectorizer.transform(contents_data)
print("A size", A.shape)

単語のインデックスを見てみましょう

In [ ]:
print("単語の数: ", len(vectorizer.get_feature_names()))
vectorizer.get_feature_names()[:10]

後で参照するので、単語のインデックスを格納しておきます。

In [ ]:
all_vocab_ar = np.array(vectorizer.get_feature_names())

## 実際にLDAを適用してみよう (Scikit-learnを使った例）

In [ ]:
model1 = LatentDirichletAllocation(n_topics=20,
                                   doc_topic_prior=0.1,
                                   topic_word_prior=0.1,
                                   max_iter=20,
                                   learning_method='online',
                                   learning_offset=50.,
                                   random_state=1234)

In [ ]:
model1.fit(A)

まずトピック x 単語を見てみましょう

In [ ]:
normalize_components = model1.components_ / model1.components_.sum(axis=0)

In [ ]:
# http://scikit-learn.org/stable/auto_examples/applications/
# topics_extraction_with_nmf_lda.html　より
n_top_words = 20
for topic_idx, topic in enumerate(normalize_components):
    print('Topic #%d:' % topic_idx)
    print(' '.join([all_vocab_ar[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

文書 x トピック行列側も見てみましょう。

In [ ]:
doc_topic_data = model1.transform(A)
doc_topic_data

scikit-learnのLDAはどうやら正規化されていないため、正規化した上で、1つ目の文書がどのトピックから来ている単語が多いのかを見てみましょう。

In [ ]:
normalize_doc_topic_data = \
 doc_topic_data / doc_topic_data.sum(axis=1, keepdims=True)

In [ ]:
for topic_idx, prob in enumerate(normalize_doc_topic_data[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

In [ ]:
loglikelihood = model1.score(A)
ppl = model1.perplexity(A)
print('対数尤度: ', loglikelihood)
ppl2 = np.exp((-1.0 * loglikelihood)/A.sum())
print('Perplexity: ', ppl)
print('Perplexity by manual: ', ppl2)

テストデータに当てはめてみましょう。

<h2> LDAを適用してみよう (ldaパッケージを使った場合)</h2> 

In [ ]:
model2 = lda.LDA(n_topics=20, n_iter=1500, random_state=1, alpha=0.5, eta=0.5)

In [ ]:
model2.fit(A)

In [ ]:
topic_word = model2.topic_word_
n_top_words = 20
for topic_idx, topic in enumerate(topic_word):
    print('Topic #%d:' % topic_idx)
    print(' '.join([all_vocab_ar[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

今回も精度として対数尤度を見てみましょう

In [ ]:
loglik = model2.loglikelihood()
print("対数尤度: ",loglik)
ppl3 = np.exp(-1.0*loglik/A.sum())
print("Perplexity: ", ppl3)

文書 x トピック行列を見てみましょう。

In [ ]:
doc_topic_data2 = model2.transform(A)
for topic_idx, prob in enumerate(doc_topic_data2[0]):
    print('Topic #%d: probality: %f' % (topic_idx, prob))

<h2>ゲーム: 文書を作ってみよう</h2>

In [ ]:
alpha = 0.1
K = 4
sampled_probs_topic = np.random.dirichlet([alpha for i in range(K)])
for i, prob in enumerate(sampled_probs_topic):
    print('トピック サイコロ %d面  確率: %.2f'%(i+1, prob))

In [ ]:
sampled_probs_topic

どのトピックから単語を選ぶか？

In [ ]:
np.random.multinomial(1, sampled_probs_topic)

単語を選ぼう！

In [ ]:
word_count = 4
beta = 0.1
sampled_probs_word = np.random.dirichlet([beta for i in range(word_count)])
for i, prob in enumerate(sampled_probs_word):
    print('単語 サイコロ %d面  確率: %.2f'%(i+1, prob))

In [ ]:
np.random.multinomial(1, sampled_probs_word)

## ユーザークラスタリングでLDAを使ってみよう

データを読み込みます

In [ ]:
with open("dataset/user_follow_topic.json", "r") as f:
    user_data_json = json.load(f)

user_id = 1000252の人はどんなトピックをフォローしているのだろう

In [ ]:
user_data_json['1000252']

対象ユーザー数を把握しておきましょう（あまりに多い場合はサンプリングする）

In [ ]:
print("ユーザー人数: ", len(user_data_json.keys()))

LDAに投入するためのデータを作りましょう

In [ ]:
user_data = []
user_id_list = []
for user_id in user_data_json.keys():
    each_user = ','.join(user_data_json[user_id])
    user_data.append(each_user)
    user_id_list.append(user_id)

In [ ]:
def tokenizer(s):
    return s.split(',')

vectorizer = CountVectorizer(tokenizer=tokenizer)
vectorizer.fit(user_data)
U = vectorizer.transform(user_data)
print("U size", U.shape)

トピックを別のリストとして格納しておきます（後々参照するため）

In [ ]:
topics = np.array(vectorizer.get_feature_names())

In [ ]:
topics[:10]

LDAを適用させます。

In [ ]:
model1 = LatentDirichletAllocation(n_topics=20,
                                   doc_topic_prior=0.1,
                                   topic_word_prior=0.1,
                                   max_iter=20,
                                   learning_method='online',
                                   learning_offset=50.,
                                   random_state=1234,
                                   n_jobs = -1 )

In [ ]:
model1.fit(U)

各トピックを理解しましょう

In [ ]:
normalize_components = model1.components_ / model1.components_.sum(axis=0)

In [ ]:
n_top_words = 20
topic_label = []
for topic_idx, topic in enumerate(normalize_components):
    topic_ = 'Topic #%d:' % topic_idx
    topic_label.append(topic_)
    print(topic_)
    print(' '.join([topics[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

ユーザー別にどんなトピックをフォローしているのかを理解しましょう。

In [ ]:
user_topic_data = model1.transform(U)

In [ ]:
normalize_user_topic_data = \
 user_topic_data / user_topic_data.sum(axis=1, keepdims=True)

In [ ]:
normalize_user_topic_data_df = pd.DataFrame(normalize_user_topic_data, columns=topic_label)

In [ ]:
normalize_user_topic_data_df.head()

### クラスタリング
一番大きいトピックをそのユーザーのクラスター番号にするという考え方でユーザーを分類します。

In [ ]:
def get_cluser_id(topic_prob_by_user):
    cluster_id = np.argmax(topic_prob_by_user)
    return cluster_id

各ユーザーごとのクラスター番号、クラスターごとのユーザー人数を調べます。

In [ ]:
user_id_with_clutser_id = {}
user_num_by_cluster_dict = {}
for i in range(len(normalize_user_topic_data)):
    
    topic_prob_by_user = normalize_user_topic_data[i]
    cluser_id = get_cluser_id(topic_prob_by_user)
    user_id = user_id_list[i]
    user_id_with_clutser_id[user_id] = cluser_id
    
    if cluser_id not in user_num_by_cluster_dict:
        user_num_by_cluster_dict[cluser_id] = 1
    else:
        user_num_by_cluster_dict[cluser_id] += 1

ユーザーごとの人数を可視化するためにpandas.DataFrameに持って行きます。

In [ ]:
user_num_by_cluster_list = []
for cluser_id, user_num in user_num_by_cluster_dict.items():
    user_num_by_cluster_list.append([cluser_id, user_num])

In [ ]:
user_num_by_cluster_df = pd.DataFrame(user_num_by_cluster_list, columns=['cluser_id', 'user_num'])

可視化するために構成比にしておきましょう（その方がわかりやすい）

In [ ]:
user_num_by_cluster_df['portion'] = user_num_by_cluster_df['user_num']/sum(user_num_by_cluster_df['user_num'])

In [ ]:
user_num_by_cluster_df

In [ ]:
user_num_by_cluster_df.plot(x='cluser_id', y='portion',  kind="bar")